In [ ]:
# seq= "TGTTGCAGGTTGGGAAGGCGAGTCAGTGGGCAGCATTCCCATCTCCAGCTTGTACAAATCCCTGCATCCCATCTCCAGCCCTGCCGAAACCAGGCAGGGTAGCTCCTGTTTTGTAATCCTGCTGCTCAGGAAGTGGAGGCAGGAGGATCGGAAGTTCAAGGTCATTCCAGTC"
# new_seq="TGTTGCAGGTTGGGAAGGCGAGTCAGTGGGCAGCATTCCCATCTCCAGCTCCCCCTCCCCCGACTCGATTGTACGTGAAGCTCCCCCTCCCCCGATCAAGCACACAAAAAACCAACACACAGATCTAATGAAAATAAAGATCTTTTATTTGTACAAATCCCTGCATCCCATCTCCAGCCCTGCCGAAACCAGGCAGGGTAGCTCCTGTTTTGTAATCCTGCTGCTCAGGAAGTGGAGGCAGGAGGATCGGAAGTTCAAGGTCATTCCAGTC"
# inserted_seq="CCCCCTCCCCCGACTCGATTGTACGTGAAGCTCCCCCTCCCCCGATCAAGCACACAAAAAACCAACACACAGATCTAATGAAAATAAAGATCTTTTATT"

# # Open the modified mm10 FASTA file
# with open(genome_file, "r") as fasta_file:
#     # Use SeqIO.parse to read the sequences from the file
#     sequences = SeqIO.parse(fasta_file, "fasta")

#     # Loop through each sequence in the file
#     for seq_record in sequences:
#         # Check if this is the sequence you modified
#         if seq_record.id == "2":
#             # Get the length of the inserted sequence
#             insertion_length = len(inserted_seq)

#             # Open the GTF file
#             with open(gtf_file, "r") as gtf_file:
#                 # Use GFF.parse to read the features from the file
#                 features = list(GFF.parse(gtf_file))

#                 # Loop through each feature in the file
#                 for feature in features:
#                     # Check if this feature overlaps with the insertion site
#                     if feature.seqid == seq_record.id and feature.end >= insertion_site:
#                         # Calculate the new start and end positions
#                         new_start = feature.start + insertion_length
#                         new_end = feature.end + insertion_length

#                         # Update the feature with the modified location
#                         feature.location = FeatureLocation(new_start, new_end, feature.strand)

#             # Write the modified GTF file
#             with open("modified_mm10.gtf", "w") as output_file:
#                 GFF.write(features, output_file)
#         else:
#             # If this is not the modified sequence, copy it to the new GTF file unmodified
#             with open("mm10.gtf", "r") as gtf_file:
#                 with open("modified_mm10.gtf", "a") as output_file:
#                     output_file.write(gtf_file.read())


In [2]:
import re
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import FeatureLocation
from BCBio import GFF
import pandas as pd


In [3]:
genome_file = "mm10_genome/mm10-2020-A-reference-sources/Mus_musculus.GRCm38.dna.primary_assembly.fa"
gtf_file = "mm10_genome/mm10-2020-A-reference-sources/gencode.vM23.primary_assembly.annotation.gtf"
output_gtf_file = "updated_mm10.gtf"
output_genome = "updated_mm10.fa"


seq= "TGTACAAATCCCTGCATCCCATCTCCAGCCCTGCCGAAACCAGGCAGGGTAGCTCCTGTTTTGTAATCCTGCTGCTCAGGAAGTGGAGGCAGGAGGATCGGAAGTTCAAGGTCATTCCAGTC"
new_seq="TGTTGCAGGTTGGGAAGGCGAGTCAGTGGGCAGCATTCCCATCTCCAGCTCCCCCTCCCCCGACTCGATTGTACGTGAAGCTCCCCCTCCCCCGATCAAGCACACAAAAAACCAACACACAGATCTAATGAAAATAAAGATCTTTTATTTGTACAAATCCCTGCATCCCATCTCCAGCCCTGCCGAAACCAGGCAGGGTAGCTCCTGTTTTGTAATCCTGCTGCTCAGGAAGTGGAGGCAGGAGGATCGGAAGTTCAAGGTCATTCCAGTC"
inserted_seq="CCCCCTCCCCCGACTCGATTGTACGTGAAGCTCCCCCTCCCCCGATCAAGCACACAAAAAACCAACACACAGATCTAATGAAAATAAAGATCTTTTATT"
# Open the modified mm10 FASTA file
modified_sequences=[]
with open(genome_file, "r") as fasta_file:
    # Use SeqIO.parse to read the sequences from the file
    sequences = SeqIO.parse(fasta_file, "fasta")

    # Loop through each sequence in the file
    for seq_record in sequences:
        # Check if this is the sequence you modified
        if seq_record.id == "2":
            # Get the length of the inserted sequence
            insertion_length = len(inserted_seq)
            match = re.search(seq, str(seq_record.seq))
            if match:
                # Insert the new subsequence after the pattern
                insertion_site = match.start()
                print(insertion_site)
                seq_record.seq = seq_record.seq[:insertion_site]+inserted_seq+seq_record.seq[insertion_site:]

        modified_sequences.append(seq_record)
                
    #             if seq in str(seq_record.seq):
    #             # Create a new sequence object with the modified subsequence
    #             old_sequence = seq_record.seq
    #             new_sequence = Seq(str(old_sequence).replace(seq, new_seq))

    #             # Create a new SeqRecord object with the modified sequence
    #             modified_seq_record = seq_record
    #             modified_seq_record.seq = new_sequence

    #             # Add the modified sequence to the list of modified sequences
    #             modified_sequences.append(modified_seq_record)
    #             print(seq_record.id)
    #         else:
    #             # Add the unmodified sequence to the list of modified sequences
    #             modified_sequences.append(seq_record)


    # Open the GTF file\
    
    

with open(output_genome, "w") as output_file:
    SeqIO.write(modified_sequences, output_file, "fasta")


105100639


In [32]:
gtf_file = "mm10_genome/mm10-2020-A-reference-sources/gencode.vM23.primary_assembly.annotation.gtf"
placeholder = "||"
placeholder = "||"

gtf_df = pd.read_csv(gtf_file, sep='\t', header=None, comment='#', names=["seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attribute"])
# gtf_df["attribute"] = gtf_df["attribute"].str.replace(r'[";]', placeholder)




In [36]:
chr_name = 'chr2'
position = insertion_site

chr_df = gtf_df[gtf_df['seqname'] == chr_name]
pos_df = chr_df[(chr_df['start'] <= position) & (chr_df['end'] > position)]
pos_df['end'] += insertion_length
gtf_df.update(pos_df)


pos_df = chr_df[(chr_df['start'] > position)]
pos_df['end'] += insertion_length
pos_df['start'] += insertion_length
                
gtf_df.update(pos_df)
gtf_df["start"] = gtf_df["start"].astype(int)
gtf_df["end"] = gtf_df["end"].astype(int)

# gtf_df.to_csv(output_gtf_file, sep='\t', header=None, index=False)


/tmp/ipykernel_716277/708619042.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_df['end'] += insertion_length
/tmp/ipykernel_716277/708619042.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_df['end'] += insertion_length
/tmp/ipykernel_716277/708619042.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [37]:
# save the updated DataFrame to a new GTF file
with open(output_gtf_file, "w") as out_file:
    for row in gtf_df.itertuples(index=False, name=None):
        # write the updated record to the output file
        out_file.write("\t".join(str(x) for x in row[:8]) + "\t" + row[8] + "\n")

In [ ]:
        
with open(gtf_file, "r") as gtf_file:
                    # Use GFF.parse to read the features from the file
                    gff_handle = list(GFF.parse(gtf_file))
#                     gff_features = gff_handle[0].features
                    for feature in gff_handle[34].features:
                        print(feature)
                        # Check if this feature overlaps with the insertion site
                        if feature.location.start.position >= insertion_site :
                            # Calculate the new start and end positions
                            new_start =  feature.location.start.position+ insertion_length
                            new_end = feature.location.end.position+ insertion_length
                            feature.location = FeatureLocation(new_start, new_end, feature.location.strand)

                        if feature.location.end.position > insertion_site and  feature.location.start.position < insertion_site:
                            # Calculate the new start and end positions
                            new_start =  feature.location.start.position
                            new_end =  feature.location.end.position + insertion_length
                            
                            # Create a new FeatureLocation object with the modified positions
#                             modified_location = FeatureLocation(new_start, new_end, feature.features.location.strand)

                            # Update the feature with the modified location
                            feature.location = FeatureLocation(new_start, new_end, feature.location.strand)

                # Create a new SeqRecord object with the modified features
# modified_seq_record = SeqRecord(seq_record.seq, id=seq_record.id, name=seq_record.name, description=seq_record.description, features=modified_features)

                # Write the modified sequence to a new GTF file
with open(output_gtf_file, "w") as output_file:
                    GFF.write(gff_handle, output_file)
#         else:
#             # If this is not the modified sequence, copy it to the new GTF file unmodified
#             with open("mm10.gtf", "r") as gtf_file:
#                 with open("modified_mm10.gtf", "a") as output_file:
#                     output_file.write(gtf_file.read())

In [11]:
with open(output_gtf_file,'r') as gtf_file:
    records = list(GFF.parse(gtf_file))

for record in records:
    for gene in record.features:
        if gene.type == 'gene':
            for transcript in gene.sub_features:
                if transcript.type == 'transcript':
                    for exon in transcript.sub_features:
                        if exon.type == 'exon':
                            # Do something with the exon feature
                            print(exon)
                            break

In [9]:
output_gtf_file = "mm10_genome/mm10-2020-A-reference-sources/gencode.vM23.primary_assembly.annotation.gtf"

with open(output_gtf_file, "r") as gtf_file:
                    # Use GFF.parse to read the features from the file
                    gff_handle = list(GFF.parse(gtf_file,limit_info={"exon":0}))
#                     gff_features = gff_handle[0].features
                    for feature in gff_handle[34].features:
                        print(feature.type)


AttributeError: 'set' object has no attribute 'items'

In [3]:

with open('updated_mm10.gtf', "r") as gtf_file:
                    # Use GFF.parse to read the features from the file
                    gff_handle = list(GFF.parse(gtf_file))
# #                     gff_features = gff_handle[0].features
#                     for feature in gff_handle[0].features:
#                         print(feature.location.end)


FileNotFoundError: [Errno 2] No such file or directory: 'updated_mm10.gtf'

In [6]:
for feature in gff_handle[0].features:
                        print(feature)

type: gene
location: [9123:58882](-)
qualifiers:
    Key: gene_id, Value: ['ENSMUSG00000079800.2']
    Key: gene_name, Value: ['AC125149.3']
    Key: gene_type, Value: ['protein_coding']
    Key: level, Value: ['3']
    Key: source, Value: ['ENSEMBL']

type: gene
location: [108389:110303](-)
qualifiers:
    Key: gene_id, Value: ['ENSMUSG00000095092.1']
    Key: gene_name, Value: ['AC125149.5']
    Key: gene_type, Value: ['protein_coding']
    Key: level, Value: ['3']
    Key: source, Value: ['ENSEMBL']

type: gene
location: [123791:124928](+)
qualifiers:
    Key: gene_id, Value: ['ENSMUSG00000079192.2']
    Key: gene_name, Value: ['AC125149.1']
    Key: gene_type, Value: ['protein_coding']
    Key: level, Value: ['3']
    Key: source, Value: ['ENSEMBL']

type: gene
location: [135394:136519](-)
qualifiers:
    Key: gene_id, Value: ['ENSMUSG00000079794.2']
    Key: gene_name, Value: ['AC125149.2']
    Key: gene_type, Value: ['protein_coding']
    Key: level, Value: ['3']
    Key: source,

In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq

# Open the mm10 FASTA file
with open("mm10_genome/mm10-2020-A-reference-sources/Mus_musculus.GRCm38.dna.primary_assembly.fa", "r") as fasta_file:
    # Use SeqIO.parse to read the sequences from the file
    sequences = SeqIO.parse(fasta_file, "fasta")

    # Create an empty list to store the modified sequences
    modified_sequences = []

    # Loop through each sequence in the file
    for seq_record in sequences:
        # Check if the subsequence is in the sequence
        if "TGTTGCAGGTTGGGAAGGCGAGTCAGTGGGCAGCATTCCCATCTCCAGCTTGTACAAATCCCTGCATCCCATCTCCAGCCCTGCCGAAACCAGGCAGGGTAGCTCCTGTTTTGTAATCCTGCTGCTCAGGAAGTGGAGGCAGGAGGATCGGAAGTTCAAGGTCATTCCAGTC" in str(seq_record.seq):
            # Create a new sequence object with the modified subsequence
            old_sequence = seq_record.seq
            new_sequence = Seq(str(old_sequence).replace(seq, new_seq))

            # Create a new SeqRecord object with the modified sequence
            modified_seq_record = seq_record
            modified_seq_record.seq = new_sequence

            # Add the modified sequence to the list of modified sequences
            modified_sequences.append(modified_seq_record)
            print(seq_record.id)
        else:
            # Add the unmodified sequence to the list of modified sequences
            modified_sequences.append(seq_record)

# Write the modified sequences to a new FASTA file
with open("modified_mm10.fa", "w") as output_file:
    SeqIO.write(modified_sequences, output_file, "fasta")


In [14]:
from Bio import SeqIO
from Bio.Seq import Seq
new_seq="TGTTGCAGGTTGGGAAGGCGAGTCAGTGGGCAGCATTCCCATCTCCAGCTCCCCCTCCCCCGACTCGATTGTACGTGAAGCTCCCCCTCCCCCGATCAAGCACACAAAAAACCAACACACAGATCTAATGAAAATAAAGATCTTTTATTTGTACAAATCCCTGCATCCCATCTCCAGCCCTGCCGAAACCAGGCAGGGTAGCTCCTGTTTTGTAATCCTGCTGCTCAGGAAGTGGAGGCAGGAGGATCGGAAGTTCAAGGTCATTCCAGTC"

# Open the mm10 FASTA file
with open("updated_mm10.fa", "r") as fasta_file:
    # Use SeqIO.parse to read the sequences from the file
    sequences = SeqIO.parse(fasta_file, "fasta")

    # Create an empty list to store the modified sequences

    # Loop through each sequence in the file
    for seq_record in sequences:
        # Check if the subsequence is in the sequence
#         if "TGTTGCAGGTTGGGAAGGCGAGTCAGTGGGCAGCATTCCCATCTCCAGCTTGTACAAATCCCTGCATCCCATCTCCAGCCCTGCCGAAACCAGGCAGGGTAGCTCCTGTTTTGTAATCCTGCTGCTCAGGAAGTGGAGGCAGGAGGATCGGAAGTTCAAGGTCATTCCAGTC" in str(seq_record.seq):
        if new_seq in str(seq_record.seq):
            print('woow')
            print(seq_record.id)

woow
2


In [ ]:
sequence = new_seq
# Open the FASTA file and search for the sequence
with open("modified_mm10.fa", "r") as f:
    header, seq = None, ""
    for line in f:
        if line.startswith(">"):
            if seq.upper().find(sequence) != -1:
                print(header)
            header = line.strip()
            seq=""
        else:
            if header!= None:
                seq += line.strip()
#         if seq.upper().find(sequence) != -1:
#             print(header)
            

In [ ]:
sequence[105100761,105100790]

In [ ]:
sequence = "TGTTGCAGGTTGGGAAGGCGAGTCAGTGGGCAGCATTCCCATCTCCAGCTTGTACAAATCCCTGCATCCCATCTCCAGCCCTGCCGAAACCAGGCAGGGTAGCTCCTGTTTTGTAATCCTGCTGCTCAGGAAGTGGAGGCAGGAGGATCGGAAGTTCAAGGTCATTCCAGTC"

if seq.upper().find(sequence) != -1:
            print(header)

In [ ]:

# Open the FASTA file and search for the sequence
with open(genome_file, "r") as f:
    header, seq = None, ""
    for line in f:
        line.strip()
        if line.startswith(">"):
            print(line.strip())
            if seq.upper().find(sequence) != -1:
                print(header)
                break
            header, seq = line.strip(), ""
        else:
            seq += line.strip()
    if seq.upper().find(sequence) != -1:
        print(header)